In [2]:
import duckdb

In [3]:
conn = duckdb.connect()

In [4]:
duckdb.sql(
    """SELECT url as game_url,
                time_control as time_control,
                rated as rated,
                time_class as time_class,
                rules as rules,
                white.rating as white_rating,
                white.result as white_result,
                black.rating as black_rating,
                black.result as black_result,    
                REGEXP_EXTRACT(pgn, '\[Event "(.*?)"', 1) as pgn_event,
                REGEXP_EXTRACT(pgn, '\[Site "(.*?)"', 1) as pgn_site,
                REGEXP_EXTRACT(pgn, '\[Round "(.*?)"', 1) as pgn_date, 
                REGEXP_EXTRACT(pgn, '\[White "(.*?)"', 1) as pgn_white_user,   
                REGEXP_EXTRACT(pgn, '\[Black "(.*?)"', 1) as pgn_black_user,
                REGEXP_EXTRACT(pgn, '\[Result "(.*?)"', 1) as pgn_result,
                REGEXP_EXTRACT(pgn, '\[CurrentPosition "(.*?)"', 1) as pgn_current_position,
                REGEXP_EXTRACT(pgn, '\[Timezone "(.*?)"', 1) as pgn_timezone,
                REGEXP_EXTRACT(pgn, '\[ECO "(.*?)"', 1) as pgn_eco,
                REGEXP_EXTRACT(pgn, '\[ECOUrl "(.*?)"', 1) as pgn_eco_url,
                REGEXP_EXTRACT(pgn, '\[StartTime "(.*?)"', 1) as start_time,
                REGEXP_EXTRACT(pgn, '\[EndTime "(.*?)"', 1) as End_time,
           FROM read_json_auto('Data/*/*.json')"""
           )



┌──────────────────────────────────────────────┬──────────────┬─────────┬────────────┬─────────┬──────────────┬──────────────┬──────────────┬──────────────┬────────────┬───────────┬──────────┬─────────────────┬────────────────┬────────────┬───────────────────────────────────────────────────────────────┬──────────────┬─────────┬───────────────────────────────────────────────────────────────────────────────────────────────┬────────────┬──────────┐
│                   game_url                   │ time_control │  rated  │ time_class │  rules  │ white_rating │ white_result │ black_rating │ black_result │ pgn_event  │ pgn_site  │ pgn_date │ pgn_white_user  │ pgn_black_user │ pgn_result │                     pgn_current_position                      │ pgn_timezone │ pgn_eco │                                          pgn_eco_url                                          │ start_time │ End_time │
│                   varchar                    │   varchar    │ boolean │  varchar   │ varchar │    

In [ ]:
# Create Dimension tables for Openings, Date, Player and GameType

# Openings Dimension Table
conn.execute("""
CREATE TABLE IF NOT EXISTS Openings AS
SELECT DISTINCT
    pgn_eco AS opening_code,
    pgn_eco_url AS opening_url
FROM read_json_auto('Data/*/*.json')
""")

# Date Dimension Table
conn.execute("""
CREATE TABLE IF NOT EXISTS Date AS
SELECT DISTINCT
    pgn_date AS game_date,
    start_time,
    End_time
FROM read_json_auto('Data/*/*.json')
""")

# Player Dimension Table
conn.execute("""

# Openings Dimension Table
conn.execute("""
CREATE TABLE IF NOT EXISTS Openings AS
SELECT DISTINCT
    pgn_eco AS opening_code,
    pgn_eco_url AS opening_url
FROM read_json_auto('Data/*/*.json')
""")

# Date Dimension Table
conn.execute("""
CREATE TABLE IF NOT EXISTS Date AS
SELECT DISTINCT
    pgn_date AS game_date,
    start_time,
    End_time
FROM read_json_auto('Data/*/*.json')
""")

# Player Dimension Table
conn.execute("""
